In [6]:
import json
import numpy as np
import cv2

with open('valid.json') as f:
    data = json.load(f)
    
storage_client = storage.Client("pose_estimation")
bucket = storage_client.get_bucket('pose_estimation_datasets')
img_name = data[0]['image']
blob = bucket.blob('MPII/images/' +  img_name)
blob.content_type = 'image/jpeg'
image = np.asarray(bytearray(blob.download_as_string()))
img = cv2.imdecode(image, cv2.IMREAD_UNCHANGED)

kpt = np.asarray(data[0]['joints'], dtype=np.int32)
if img.shape[0] != 960 or img.shape[1] != 720:
    kpt[:,0] = kpt[:,0] * (960/img.shape[1])
    kpt[:,1] = kpt[:,1] * (720/img.shape[0])

In [39]:
def gaussian_kernel(h, w, x, y, sigma, stride):
    h, w, x, y = h//stride, w//stride, x//stride, y//stride
    ycoords, xcoords = np.mgrid[0:h, 0:w]
    num = -1 * (np.square(ycoords - y) + np.square(xcoords - x))
    den = 2 * np.square(sigma)
    ans = np.exp(num/den)
    return ans/np.sum(ans)

In [40]:
stride = 1
sigma = 3
heatmap = np.zeros((720//stride, 960//stride, len(kpt)))
for i in range(len(kpt)):
    heatmap[:, :, i] = gaussian_kernel(720, 960, kpt[i][0], kpt[i][1], sigma, stride)

In [41]:
gaussian_kernel(3, 3, 1, 2, 3, 1)

array([[0.0953599 , 0.1008076 , 0.0953599 ],
       [0.11265441, 0.11909011, 0.11265441],
       [0.11909011, 0.12589345, 0.11909011]])